## setup

In [3]:
## Install EVA package if needed
# %pip install "evadb[dev]" --quiet

import os
import time
from psutil import process_iter
from signal import SIGTERM
import re
import itertools

def connect_to_server():
    from eva.server.db_api import connect
    %pip install nest_asyncio --quiet
    import nest_asyncio
    nest_asyncio.apply()

    # Connect client with server
    connection = connect(host = '127.0.0.1', port = 5432) 
    cursor = connection.cursor()

    return cursor

In [4]:
cursor = connect_to_server()

Note: you may need to restart the kernel to use updated packages.


In [3]:
cursor.execute('DROP TABLE ObjectDetectionVideos')
response = cursor.fetch_all()
print(response)
cursor.execute('LOAD VIDEO "../data/ua_detrac/ua_detrac.mp4" INTO ObjectDetectionVideos;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                    0
0  Table Successfully dropped: ObjectDetectionVideos
@query_time: 0.05822162499999983
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.05276537499999989


In [4]:
cursor.execute('DROP TABLE MNIST')
response = cursor.fetch_all()
print(response)
cursor.execute('LOAD VIDEO "../data/mnist/mnist.mp4" INTO MNIST;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Table: MNIST does not exist
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.06429687499996817


In [5]:
cursor.execute('DROP UDF GaussianBlur')
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS GaussianBlur
    INPUT (frames NDARRAY UINT8(3, ANYDIM, ANYDIM))
    OUTPUT (blurred_frame_array NDARRAY UINT8(3, ANYDIM, ANYDIM))
    TYPE ndarray
    IMPL 'udfs/ndarray/gaussian_blur.py'
    """)

response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                        0
0  UDF GaussianBlur successfully dropped
@query_time: 0.022498833000000218
@status: ResponseStatus.SUCCESS
@batch: 
                                                       0
0  UDF GaussianBlur successfully added to the database.
@query_time: 0.02127783299999919


## applying simple image transformations on videos ##

In [11]:
# overwrites traffic video by applying simple image transformations such as Gaussian Blur

cursor.execute('OVERWRITE MNIST BY GaussianBlur(data)')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                  0
0  Table successfully overwritten by: GaussianBlur
@query_time: 0.2340028750000016


## annotate frames with detected objects on videos ##

In [ ]:
# overwrites traffic video by annotating objects detected by YoloV5

cursor.execute('OVERWRITE ObjectDetectionVideos BY Annotate(data, YoloV5(data))')
response = cursor.fetch_all()
print(response)

In [ ]:
cursor.execute("""SELECT * FROM ObjectDetectionVideos WHERE id < 20""")
response = cursor.fetch_all()
response.as_df()

## apply simple image transformations on images

In [6]:
cursor.execute('DROP TABLE EMOTIONS')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/emotion_detector/happy.jpg" INTO EMOTIONS;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/emotion_detector/angry.jpg" INTO EMOTIONS;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                       0
0  Table Successfully dropped: EMOTIONS
@query_time: 0.05122599999999977
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.03948762499999958
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.012955124999997736


In [7]:
cursor.execute('SELECT * FROM EMOTIONS;')
response = cursor.fetch_all()
response.as_df()

,emotions.name,emotions.data
0,../data/emotion_detector/happy.jpg,"[[[12, 255, 0], [4, 254, 0], [4, 255, 0], [1, ..."
1,../data/emotion_detector/angry.jpg,"[[[145, 161, 173], [145, 161, 173], [145, 161,..."


In [8]:
cursor.execute('OVERWRITE EMOTIONS BY GaussianBlur(data);')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                  0
0  Table successfully overwritten by: GaussianBlur
@query_time: 0.09478745799999899


In [9]:
cursor.execute('SELECT * FROM EMOTIONS;')
response = cursor.fetch_all()
response.as_df()

,emotions.name,emotions.data
0,../data/emotion_detector/modified_happy.jpg,"[[[0, 255, 0], [0, 251, 0], [9, 245, 13], [24,..."
1,../data/emotion_detector/modified_angry.jpg,"[[[147, 162, 171], [147, 162, 171], [147, 162,..."
